In [1]:
import numpy as np
from scipy import signal
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from matplotlib import cm
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from data_module import DataModule
import pytorch_lightning as pl
from models import UNet, WeightedLoss
from pl_trainer import *
from utils import *
from pytorch_lightning.callbacks import ModelCheckpoint


In [ ]:
class Ploter(object):
    def __init__(self, a=1, h=0.01, path='./u/sin/'):
        N = round(a/h)
        x = np.linspace(0, a, N)
        y = np.linspace(0, a, N)        
        self.xx, self.yy = np.meshgrid(x, y)
        self.pres = list(Path(path).glob("Test_u_*"))
#         print(self.pres)
        
    def draw(self, levels, n=10):
        for i in range(n):
            fig = plt.figure()
            fig.set_figheight(10)
            fig.set_figwidth(20)
            
            pre = np.load(self.pres[i])
            
            ax1 = fig.add_subplot(1, 2, 1, projection='3d')
            ax2 = fig.add_subplot(1, 2, 2)
            
            surf_pre = ax1.plot_surface(self.xx, self.yy, pre, cmap=cm.coolwarm, linewidth=0, antialiased=False)
#             surf_ans = ax2.plot_surface(self.xx, self.yy, ans, cmap=cm.coolwarm, linewidth=0, antialiased=False)
            
            cp1 = ax2.contour(self.xx, self.yy, pre, colors='b', linewidths=2)
            ax2.clabel(cp1, inline=True, fontsize=10)
#             cp2 = ax3.contour(self.xx, self.yy, ans, colors='r', levels=levels, linewidths=3, linestyles='solid')
            fig.tight_layout()

            
plotter = Ploter(h=0.01)
plotter.draw(levels=[0,0.2, 0.4, 0.6, 0.8, 1, 1.2, 1.4, 1.6, 1.8, 2])

In [2]:
from pl_trainer import *
class TestSin(object):
    def __init__(self, a, h, model, ckpt_path, u, f, Q):
        N = round(a/h)
        x = np.linspace(0, a, N)
        y = np.linspace(0, a, N)        
        self.xx, self.yy = np.meshgrid(x, y)
        ckpt = torch.load(ckpt_path)
        self.model = model
        self.model.load_state_dict(ckpt['state_dict'])
        self.model.freeze()
        self.u = u
        self.f = f
        self.Q = Q
        force = self.f(self.xx, self.yy)
        self.data = np.concatenate((self.xx[np.newaxis,...], self.yy[np.newaxis,...], force[np.newaxis,...]))
        self.label = self.u(self.xx, self.yy)
        
    def _mul_q(self, q):
        tensor = torch.from_numpy(self.data).to(dtype=torch.float32)
        tensor[2,...] *= q
        return tensor[None, ...]
        
    def draw(self, levels):
        
        for q in self.Q:
            data = self._mul_q(q)
            ans = q * self.label
            pre = self.model(data).squeeze().numpy()
            
            print(np.linalg.norm(pre-ans))
            fig = plt.figure()
            fig.set_figheight(9)
            fig.set_figwidth(27)
            
            ax1 = fig.add_subplot(1, 3, 1, projection='3d')
            ax2 = fig.add_subplot(1, 3, 2, projection='3d')
            ax3 = fig.add_subplot(1, 3, 3)
#             ax4 = fig.add_subplot(1, 3, 2, projection='3d')
            
    
            ax1.set_title(f'$Q={q:.4f}$')
            ax2.set_title(f'$Q={q:.4f}$')
            
            surf_pre = ax1.plot_surface(self.xx, self.yy, pre, cmap=cm.coolwarm, linewidth=0, antialiased=False)
            surf_ans = ax2.plot_surface(self.xx, self.yy, np.abs(ans-pre), cmap=cm.coolwarm, linewidth=0, antialiased=False)
            
            cp1 = ax3.contour(self.xx, self.yy, pre, colors='b', levels=levels, linewidths=2)
            cp2 = ax3.contour(self.xx, self.yy, ans, colors='r', levels=levels, linewidths=3, linestyles='solid')
            ax3.clabel(cp1, inline=True, fontsize=10)
            ax3.clabel(cp2, inline=True, fontsize=10)
            
#             fig.tight_layout()


model = Hard_FD_Rec_Module(net = UNet(1, 3, 2),
                           loss = WeightedLoss(diff_fun = F.l1_loss,alpha = 10.0,beta = 0.0),
                                  fig_save_path = './u/sin/',lr = 1e-3,
                                  rhs = Dir_RHS_Rec(k=1, h=0.005, value=1))
plotter = TestSin(a=1, h=0.01, model=model, ckpt_path='./lightning_logs/version_0/checkpoints/epoch=72-step=18103.ckpt',
                  f=lambda x,y: -8*np.pi*np.pi*np.sin(2*np.pi*x)*np.sin(2*np.pi*y),
                  u=lambda x,y: np.sin(2*np.pi*x)*np.sin(2*np.pi*y),
                  Q=list(np.arange(0.5,1.5,0.1)))
plotter.draw(levels=[-0.5, -0.4, -0.3, -0.2, -0.1,0,0.1,0.2, 0.3, 0.4, 0.5])

/home/whujjq/anaconda3/lib/python3.9/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646756402876/work/c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
class ValPloter(object):
    def __init__(self, a, h, path, draw_data=False):
        self.path = path
        N = round(2*a/h)
        x = np.linspace(-a, a, N)
        y = np.linspace(-a, a, N)
        self.xx, self.yy = np.meshgrid(x, y)
        if not draw_data:
            self.us = list(Path(path).glob("Val_u*"))
#             self.us.append(path+"Weight.npy")
            self.diffs = list(Path(path).glob("Val_diff*"))
        else:
            self.us = list(Path(path).glob("*.npy"))
    
    def draw(self, levels, n=10, diff=False):
        if n > len(self.us):
            n=len(self.us)
        for i in range(n): 
            fig = plt.figure()
            if diff:
                fig.set_figheight(9)
                fig.set_figwidth(27)
                u = np.load(self.us[i])
                d = np.load(self.diffs[i])
                ax1 = fig.add_subplot(1, 3, 1, projection='3d')
                ax2 = fig.add_subplot(1, 3, 2)
                ax3 = fig.add_subplot(1, 3, 3, projection='3d')

                surfu = ax1.plot_surface(self.xx, self.yy, u, cmap=cm.coolwarm, linewidth=0, antialiased=False)
                cp = ax2.contour(self.xx, self.yy, u, colors='k', levels=levels)
                surfd = ax3.plot_surface(self.xx, self.yy, d, cmap=cm.coolwarm, linewidth=0, antialiased=False)
                ax2.clabel(cp, inline=True, fontsize=10)
                fig.tight_layout()
            else:
                fig.set_figheight(9)
                fig.set_figwidth(18)
                u = np.load(self.us[i])
                ax1 = fig.add_subplot(1, 2, 1, projection='3d')
                ax2 = fig.add_subplot(1, 2, 2)
                surfu = ax1.plot_surface(self.xx, self.yy, u, cmap=cm.coolwarm, linewidth=0, antialiased=False)
                cp = ax2.contour(self.xx, self.yy, u, colors='k', levels=levels)
                ax2.clabel(cp, inline=True, fontsize=10)
                fig.tight_layout()
                
    def draw_data(self, levels, n=5):
        for u in self.us[:n]: 
                fig = plt.figure()
                fig.set_figheight(9)
                fig.set_figwidth(18)
                z = (np.load(u)[2,:,:]).squeeze()
                ax1 = fig.add_subplot(1, 2, 1, projection='3d')
                ax2 = fig.add_subplot(1, 2, 2)
                surf = ax1.plot_surface(self.xx, self.yy, z,cmap=cm.coolwarm, linewidth=0, antialiased=False)
                cp = ax2.contour(self.xx, self.yy, z, colors='k', levels=levels)
                ax2.clabel(cp, inline=True, fontsize=10)
                fig.tight_layout()

    def plot_weight(self):
        lamda = self.path + "Weight.npy"
        lamda = np.load(lamda)
        
        im = plt.imshow(lamda, interpolation='nearest')
        plt.colorbar(im)
        
                
                
    def __len__(self):
        return len(self.us)

In [ ]:
val_plotter = ValPloter(1, 0.005, './u/case2/')
# val_plotter.draw(levels=[65, 75, 80, 85, 90, 95, 99], diff=True)

# val_plotter = ValPloter(1, 0.01, './u/case1/')
val_plotter.draw(levels=[-1.0, -0.4, -0.3, -0.2, -0.15, -0.1, -0.05], diff=True)
# val_plotter.plot_weight()
# val_plotter.draw(levels=None, diff=True)


# data_plotter = Ploter(500, 2.5, './data/case1')
# data_plotter.draw(levels=None)

# data_plotter = ValPloter(1, 0.005, './data/green/', True)
# data_plotter.draw_data(levels=None)

In [ ]:
class TestPloter(object):
    def __init__(self, real_path, a, h, k, mean, sigma, model, ckpt):
        self.reals = list(Path(real_path).glob('*.npy'))
        N = round(2*a/h)
        x = np.linspace(-a, a, N)
        y = np.linspace(-a, a, N)
        self.xx, self.yy = np.meshgrid(x, y)
        self.a = a
        self.h = h
        self.ckpt = ckpt
        self.mean = mean
        self.sigma = sigma
        self.model = model
        
        
    def triple_plot(self, levels):
        for r in self.reals:
            cap = -float(r.name[:-4])
            ans = np.load(r)
#             pre = test_g(cap = cap, a=self.a, h=self.h, ckpt_path=self.ckpt,
#                         mean=self.mean, sigma=self.sigma,model=self.model).numpy()
            pre = test(cap = cap, a=self.a, h=self.h, ckpt_path=self.ckpt,
                        loc=self.mean,model=self.model).numpy()

            fig = plt.figure()
            fig.set_figheight(9)
            fig.set_figwidth(29)
            plt.rcParams['font.size'] = '15'

            ax1 = fig.add_subplot(1, 3, 1, projection='3d')
            surf1 = ax1.plot_surface(self.xx, self.yy, ans, cmap=cm.Spectral, linewidth=0, antialiased=False)
            ax1.set_title(f'$REAL:Q={cap:.2f}$')C

            ax2 = fig.add_subplot(1, 3, 2, projection='3d')
            surf1 = ax2.plot_surface(self.xx, self.yy, pre, cmap=cm.Spectral, linewidth=0, antialiased=False)
            ax2.set_title(f'$PRE:Q={cap:.2f}$')

            ax3 = fig.add_subplot(1, 3, 3)

            cp1 = ax3.contour(self.xx, self.yy, pre, colors='r', levels=levels, linewidths=2)
            cp2 = ax3.contour(self.xx, self.yy, ans, colors='b', levels=levels, linewidths=3, linestyles='solid')
            ax3.clabel(cp1, fmt='%.2f', inline=1, fontsize='large')
            ax3.clabel(cp2, fmt='%.2f', inline=1, fontsize='large')
            
    def fourth_plot(self, levels):
        pres = []
        reals = []
        caps = []
        
        for r in self.reals:
            cap = -float(r.name[:-4])
            caps.append(cap)
            reals.append(np.load(r))
            pres.append(test_g(cap = cap, a=self.a, h=self.h, ckpt_path=self.ckpt, mean=self.mean, sigma=self.sigma,
                              model=self.model).numpy())
        
        fig1 = plt.figure()
        fig1.set_figheight(20)
        fig1.set_figwidth(20)
        
        for i in range(4):
            ax = fig1.add_subplot(2,2, i+1, projection='3d')
            surf = ax.plot_surface(self.xx, self.yy, pres[i],cmap=cm.Spectral,linewidth=0, antialiased=False)
            ax.set_title(f'$Q=-{caps[i]:.2f}$',fontsize=25)
            fig1.colorbar(surf, ax=ax, shrink=0.5)
        
        fig2 = plt.figure()
        fig2.set_figheight(20)
        fig2.set_figwidth(20)
        
        for i in range(4):
            ax = fig2.add_subplot(2,2,i+1)
            ax.set_title(f'$Q=-{caps[i]:.2f}$',fontsize=25)
            cp1 = ax.contour(self.xx, self.yy, pres[i], colors='r', levels=levels, linewidths=2)
            cp2 = ax.contour(self.xx, self.yy, reals[i], colors='b', levels=levels, linewidths=3, linestyles='solid')
            ax.clabel(cp1, fmt='%.2f', inline=1, fontsize='large')
            ax.clabel(cp2, fmt='%.2f', inline=1, fontsize='large')


In [ ]:
model = FiniteDiffModule( net = UNet(out_channels = 1,in_channels = 3,factor = 1),
                                    loss = WeightedLoss(diff_fun = F.l1_loss,alpha = 10.0,beta = 0.0),
                                    fig_save_path = './u/case2/',
                                    lr = 1e-3,
                                    rhs = Dir_RHS(k=1, h=0.01, value=0))
plotter = TestPloter(real_path = './test_data/green/',a=1, h=0.01, k=1, mean=[0,0], sigma=[[1e-3,0],[0,1e-3]],model=model, 
                     ckpt= './lightning_logs/version_14/checkpoints/epoch=49-step=24749.ckpt')
plotter.triple_plot([-1.0, -0.4, -0.3, -0.2, -0.15, -0.1, -0.05])

In [ ]:
model = FiniteDiffModuleSoft( net = UNet(out_channels = 1,in_channels = 3,factor = 1),
                                    loss = WeightedLoss(diff_fun = F.l1_loss,alpha = 10.0,beta = 0.0),
                                    fig_save_path = './u/case2/',
                                    lr = 1e-3,
                                    rhs = Dir_RHS(k=100, h=5, value=100))
# plotter = TestPloter(real_path = './test_data/water/',a=500, h=5, k=100, mean=[0,0], sigma=[[1.56, 0],[0,1.56]],model=model, 
#                      ckpt= './lightning_logs/version_15/checkpoints/epoch=34-step=17324.ckpt')
# plotter.triple_plot(levels=[65, 75, 80, 85, 90, 95, 99])

In [ ]:
from utils import four_test
a = 1
h = 0.005
N = round(2*a/h)
x = np.linspace(-a, a, N)
y = np.linspace(-a, a, N)
xx, yy = np.meshgrid(x, y)
levels = [-1.0, -0.4, -0.3, -0.25,-0.2, -0.15, -0.1, -0.05]
pre = four_test(caps=[1.2,1.2,1.2,1.2], a=1, h=0.005, ckpt_path="./lightning_logs/version_0/checkpoints/epoch=61-step=153449.ckpt", 
                         model=model, mus=[[-0.5,-0.5],[0.5,-0.5],[-0.5,0.5],[0.5,0.5]], sigma=1e-4)
ans = np.load("./real/four.npy")
fig = plt.figure()
fig.set_figheight(9)
fig.set_figwidth(29)
plt.rcParams['font.size'] = '15'

ax1 = fig.add_subplot(1, 3, 1, projection='3d')
surf1 = ax1.plot_surface(xx, yy, -ans, cmap=cm.Spectral, linewidth=0, antialiased=False)
ax1.set_title(f'$REAL:{-(ans.max()):.2f}$')

# ax1.set_title(f'$REAL:Q={cap:.2f}$')

ax2 = fig.add_subplot(1, 3, 2, projection='3d')
surf1 = ax2.plot_surface(xx, yy, pre, cmap=cm.Spectral, linewidth=0, antialiased=False)
ax2.set_title(f'$PRE:{(pre.min()):.2f}$')

# ax2.set_title(f'$PRE:Q={cap:.2f}$')

ax3 = fig.add_subplot(1, 3, 3)

cp1 = ax3.contour(xx, yy, pre, colors='r', levels=levels, linewidths=2)
cp2 = ax3.contour(xx, yy, -ans, colors='b', levels=levels, linewidths=3, linestyles='solid')
ax3.clabel(cp1, fmt='%.2f', inline=1, fontsize='large')
ax3.clabel(cp2, fmt='%.2f', inline=1, fontsize='large')

In [ ]:
def sigmoid(x):
    return 1 / (1+np.exp(-x))
x = np.linspace(-10,10,100)
y = 10 * sigmoid(x - 5)
# y = 10 * sigmoid(-0.05*(x - 1)) 
plt.plot(x,y)
plt.show()